In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorboard
import datetime
from sklearn.model_selection import train_test_split

(x_train_full, y_train_full), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

x_train_full = x_train_full / 255
x_test = x_test / 255

x_train, x_valid, y_train, y_valid = train_test_split(x_train_full, y_train_full, random_state=42)
x_train.shape

(37500, 32, 32, 3)

In [3]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(64, 7, strides=1,
                              padding="SAME", activation="relu",
                              input_shape=[32, 32, 3]))
model.add(keras.layers.Conv2D(64, 3, strides=1, padding="SAME", activation="relu"))
model.add(keras.layers.MaxPool2D(2))
model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Conv2D(128, 3, strides=1, padding="SAME", activation="relu"))
model.add(keras.layers.Conv2D(128, 3, strides=1, padding="SAME", activation="relu"))
model.add(keras.layers.MaxPool2D(2))
model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Conv2D(256, 1, strides=1, padding="SAME", activation="relu"))
model.add(keras.layers.Conv2D(256, 1, strides=1, padding="SAME", activation="relu"))
model.add(keras.layers.MaxPool2D(2))
model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(50, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))

In [2]:
class ResidualUnit(keras.layers.Layer):
  def __init__(self, filters, strides, activation="relu", **kwargs):
    super().__init__(**kwargs)
    self.activation = activation
    self.main_layers = [
      keras.layers.Conv2D(filters, 3, strides=strides,
                          padding="SAME", use_bias=False),
      keras.layers.BatchNormalization(),
      self.activation      ,                  
      keras.layers.Conv2D(filters, 3, strides=strides,
                          padding="SAME", use_bias=False),
      keras.layers.BatchNormalization()
    ]
    skip_layers = []
    if strides > 1:
      skip_layers = [
        keras.layers.Conv2D(filters, 1, strides=strides,
                          padding="SAME", use_bias=False),
      keras.layers.BatchNormalization()                     
      ]

  def call(self, inputs):
    Z = inputs
    for layer in self.main_layers:
      Z = layer(Z)
    skip_Z = inputs
    for layer in self.skip_layers:
      skip_Z = layer(skip_Z)
    return self.activation(skip_Z + Z)

In [3]:
model = keras.Sequential()
keras.layers.Conv2D(64, 7, strides=2, input_shape=[32, 32, 3],
                      padding = "SAME", use_bias=False)
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding="SAME"))
prev_filters=64
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
  strides=1 if filters==prev_filters else 2
  model.add(ResidualUnit(filters, strides=strides))
  prev_filters = filters
model.add(keras.layers.GlobalAveragePooling2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10, activation="softmax"))

In [4]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.Nadam(),
              metrics=["accuracy"])

In [5]:
history = model.fit(x_train, y_train, epochs=50,
          validation_data=(x_valid, y_valid))

Epoch 1/50


TypeError: ignored